In [1]:
import sys, os
sys.path.insert(1, '..')

import pandas as pd
import numpy as np
from pandas.util.testing import assert_almost_equal, array_equivalent

from rdsutils.query import query_postgres

%load_ext autoreload
%autoreload 2

/home/ec2-user/anaconda3/envs/ml4risk/lib/python3.7/site-packages/ipykernel/__main__.py:6: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.


### Query Data

In [3]:
pstr_df = pd.read_parquet("s3://sofi-data-science/jxu/pl-gen4/data_dump/archive/bad-def-n-perf-window/payment_str.parquet")

In [4]:
pstr_df.head()

,id,days_past_due,pl_custom_score,bankruptcy,charge_off_date,fico_score,confirmed_fraud,deliq_30p_90days,loan_id,initial_term,...,bad90dpd,status18,status6after18,status3after18,status3after20,status20,status6after20,status6after24,status3after24,status24
0,222181,0,414.0,0,None,718,0,0,31706,5,...,0,C,C,C,C,C,C,C,C,C
1,222212,0,328.0,0,None,705,0,0,31466,3,...,0,P,P,P,P,P,P,P,P,P
2,222220,0,390.0,0,None,775,0,0,31467,7,...,0,C,C,C,C,C,C,C,C,C
3,222225,0,338.0,0,None,751,0,0,31434,7,...,0,C,C,C,C,C,C,C,C,C
4,222301,0,319.0,0,None,753,0,0,31435,7,...,0,C,C,C,C,C,C,C,C,C


### Experiment with status6after18
---

#### Old code

In [8]:
def payment_st(perf_var):
    if perf_var=='C':
        return 'CURRENT'
    elif (perf_var=='1') |( perf_var=='2'):
        return '1-29DPD'
    elif  perf_var=='3':
        return '30-60DPD'
    elif perf_var=='4':
        return '60-90DPD'
    elif perf_var=='5':
        return '90-120DPD'
    elif perf_var in ['B','D','W','6']:
        return 'Default'
    else:
        return perf_var
    
enough18=pstr_df[(~pstr_df['status18'].isin(['N','X','U','M','F'])) ]
enough18['status6after18']=enough18.apply(lambda x: x['status6after18'] if x['status6after18'] !='N' 
                                          else ('N' if pd.isnull(x.status_string) else x.status_string[-1]), axis=1)
enough18=pstr_df[(~pstr_df['status6after18'].isin(['X','U','M','F','N']))] 
enough18['status18_']=enough18['status18'].apply(lambda x: payment_st(x) )
enough18['status6after18_']=enough18['status6after18'].apply(lambda x: payment_st(x) )
old = pd.crosstab(enough18.status18_,enough18.status6after18_,margins=True)
old

/home/ec2-user/anaconda3/envs/ml4risk/lib/python3.7/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/ml4risk/lib/python3.7/site-packages/ipykernel/__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/ml4risk/lib/python3.7/site-packages/ipykernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


status6after18_,1-29DPD,30-60DPD,60-90DPD,90-120DPD,CURRENT,Default,P,All
status18_,,,,,,,,
1-29DPD,556,133,95,85,779,958,127,2733
30-60DPD,78,54,75,83,242,1198,31,1761
60-90DPD,26,32,61,77,108,1627,13,1944
90-120DPD,7,13,30,57,58,1204,7,1376
CURRENT,2060,1351,1440,1062,303141,3962,36220,349236
Default,0,0,0,0,0,15997,6,16003
F,41,32,32,27,823,79,76,1110
M,0,0,0,0,0,7,0,7
P,0,0,0,0,0,1,88251,88252


#### New code
---

In [9]:
pstr_df["status18"] = pw.get_payment_status_month_k(pstr_df["status_string"], 18)
pstr_df["status6after18"] = pw.get_payment_status_month_k(pstr_df["status_string"], 24)

tbl1 = pw.get_status_transition_table(
    pstr_df, "status18", "status6after18", margins=True
)

NameError: name 'pw' is not defined